<a href="https://colab.research.google.com/github/MeghneelG0/fahp_vikor_SRPM/blob/main/fahp_Vikor_SRPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Requirements.csv")

# Step 1: Define Criteria and Performance Scores
criteria = ['Cost', 'Value', 'Importance']
performance_scores = data[['Cost', 'Value', 'Importance']].apply(pd.to_numeric, errors='coerce').values

# Step 2: Intuitionistic Fuzzy AHP (IFAHP)
# Define intuitionistic fuzzy pairwise comparison matrix for criteria
# Each element is a tuple (membership, non-membership)
criteria_matrix = [
    [(1, 1, 1), (2, 3, 4), (5, 6, 7)],  # Cost vs Value, Cost vs Importance
    [(1/4, 1/3, 1/2), (1, 1, 1), (2, 3, 4)],  # Value vs Cost, Value vs Importance
    [(1/7, 1/6, 1/5), (1/4, 1/3, 1/2), (1, 1, 1)]  # Importance vs Cost, Importance vs Value
]

# Function to compute intuitionistic fuzzy weights
def compute_fahp_weights(matrix):
    n = len(matrix)
    fuzzy_sums = np.zeros((n, 3))  # Store (l, m, u) for each criterion

    # Step 1: Sum each column
    for j in range(n):
        for i in range(n):
            fuzzy_sums[j] += np.array(matrix[i][j])

    # Step 2: Compute Si values (row-wise sum divided by total sum)
    S_values = np.zeros((n, 3))
    for i in range(n):
        S_values[i] = np.sum(matrix[i], axis=0) / fuzzy_sums.sum(axis=0)

    # Step 3: Defuzzify using centroid method
    weights = (S_values[:, 0] + S_values[:, 1] + S_values[:, 2]) / 3
    weights /= np.sum(weights)  # Normalize to sum to 1
    return weights

# Compute IFAHP weights
fahp_weights = compute_fahp_weights(criteria_matrix)
print("FAHP Weights:", fahp_weights)

# Step 3: VIKOR Method
# Normalize performance scores
def normalize_scores(scores):
    normalized_scores = np.zeros_like(scores)
    for j in range(scores.shape[1]):
        normalized_scores[:, j] = scores[:, j] / np.sqrt(np.sum(scores[:, j]**2))
    return normalized_scores

normalized_scores = normalize_scores(performance_scores)

# Compute weighted normalized scores
weighted_scores = normalized_scores * fahp_weights

# Determine ideal and negative-ideal solutions
ideal_best = np.max(weighted_scores, axis=0)
ideal_worst = np.min(weighted_scores, axis=0)

# Compute group utility (S_i) and individual regret (R_i)
S = np.sum((ideal_best - weighted_scores) / (ideal_best - ideal_worst), axis=1)
R = np.max((ideal_best - weighted_scores) / (ideal_best - ideal_worst), axis=1)

# Compute VIKOR index (Q_i)
v = 0.5  # Weight for strategy of maximum group utility
Q = v * (S - np.min(S)) / (np.max(S) - np.min(S)) + (1 - v) * (R - np.min(R)) / (np.max(R) - np.min(R))

# Rank requirements based on Q values
rankings = np.argsort(Q) + 1  # Add 1 to convert to 1-based ranking

# Add rankings to the dataset
data['Rank'] = rankings

# Display the ranked requirements
ranked_data = data[['Req ID', 'Req Name', 'Rank']].sort_values(by='Rank')
print(ranked_data)

# Save the ranked requirements to an Excel file
ranked_data.to_excel("Ranked_Requirements.xlsx", index=False)
print("Ranked requirements saved to 'Ranked_Requirements.xlsx'.")

FAHP Weights: [0.62978244 0.27240156 0.097816  ]
    Req ID                  Req Name  Rank
0      101                       URL     1
71     172                Book Event     2
70     171               Share Event     3
69     170               Event Terms     4
68     169           View Larger Map     5
..     ...                       ...   ...
28     129  Movie's Trending Stories    96
27     128             Movie Details    97
26     127              Select Movie    98
86     187           Change Language    99
99     200                    Logout   100

[100 rows x 3 columns]
Ranked requirements saved to 'Ranked_Requirements.xlsx'.
